In [32]:
import os
import pandas as pd
from metapub import PubMedFetcher
from functools import reduce
from Bio import Entrez


In [29]:
def fetch_articles(keyword=None, num_of_articles=None):
    os.environ['NCBI_API_KEY'] = '7bcc52edf3bcb7bfc9156aa5d50e573e2309'
    fetch = PubMedFetcher()

    if keyword:
        pmids = fetch.pmids_for_query(keyword, retmax=num_of_articles)

        articles = {}
        for pmid in pmids:
            articles[pmid] = fetch.article_by_pmid(pmid)

        data = {
            'Title': [articles[pmid].title for pmid in pmids],
            'Abstract': [articles[pmid].abstract for pmid in pmids],
            'Author': [articles[pmid].authors for pmid in pmids],
            'Year': [articles[pmid].year for pmid in pmids],
            'Volume': [articles[pmid].volume for pmid in pmids],
            'Issue': [articles[pmid].issue for pmid in pmids],
            'Journal': [articles[pmid].journal for pmid in pmids],
            'Citation': [articles[pmid].citation for pmid in pmids],
            'Link': [f"https://pubmed.ncbi.nlm.nih.gov/{pmid}/" for pmid in pmids]
        }

        df = pd.DataFrame(data)
        return df
    else:
        print("Please provide a keyword to search for articles.")
        return None

In [30]:
# Example usage:
# For keyword search:
df = fetch_articles(keyword="IL-6", num_of_articles=3)
df

,Title,Abstract,Author,Year,Volume,Issue,Journal,Citation,Link
0,DNA corona on nanoparticles leads to an enhanc...,Autoimmune and inflammatory diseases are highl...,"[Anees F, Montoya DA, Pisetsky DS, Payne CK]",2024,121,11,Proc Natl Acad Sci U S A,"Anees F, et al. DNA corona on nanoparticles le...",https://pubmed.ncbi.nlm.nih.gov/38442162/
1,Exploring the interplay of psychological and b...,Ageing and chronic stress have been linked to ...,"[Souza-Talarico JN, Chesak S, Elizalde N, Liu ...",2024,None,None,Stress Health,"Souza-Talarico JN, et al. Exploring the interp...",https://pubmed.ncbi.nlm.nih.gov/38442010/
2,Asperustins A-J: Austocystins with Immunosuppr...,Ten new (,"[Chang JL, Gan YT, Zhou YH, Peng XG, Xie ZY, M...",2024,None,None,J Nat Prod,"Chang JL, et al. Asperustins A-J: Austocystins...",https://pubmed.ncbi.nlm.nih.gov/38441877/


In [42]:
def article_links(start_date, end_date='3000'):
    """
    start_date, end_date = 'YYYY/MM/DD'
    returns a list of PubMedCentral links and a 2nd list of DOI links
    """
    Entrez.email = "trevsauer@gmail.com"

    # Generate search query
    search_query = '("%s"[Date - Publication] : "%s"[Date - Publication]) ' % (start_date, end_date)
    print("Search Query:", search_query)

    # Get all articles in the specified date range
    handle = Entrez.esearch(db="pubmed", term=search_query)
    records = Entrez.read(handle)
    handle.close()

    pmc_articles = []
    doi = []

    # Check if records is a list
    if isinstance(records, list):
        for record in records:
            # Get all PMC articles
            if record.get('MedlineCitation'):
                if record['MedlineCitation'].get('OtherID'):
                    for other_id in record['MedlineCitation']['OtherID']:
                        if other_id.title().startswith('Pmc'):
                            pmc_articles.append('http://www.ncbi.nlm.nih.gov/pmc/articles/%s/pdf/' % (other_id.title().upper()))

            # Get all DOIs
            if record.get('PubmedData'):
                if record['PubmedData'].get('ArticleIdList'):
                    for other_id in record['PubmedData']['ArticleIdList']:
                        if 'doi' in other_id.attributes.values():
                            doi.append('http://dx.doi.org/' + other_id.title())
    else:
        print("No records found.")

    return pmc_articles, doi

if __name__ == '__main__':
    print(article_links('2016/12/20', '2024/02/01'))

Search Query: ("2016/12/20"[Date - Publication] : "2024/02/01"[Date - Publication]) 
No records found.
([], [])
